# NSE Options Trading Analysis
## Strategy 1: Short Straddle

Identify the mathematically best strike prices for selling both CALL and PUT options.

In [ ]:
# Import required libraries
from nse_api import NSEDataFetcher
import pandas as pd
from time import sleep
from IPython.display import clear_output
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt

# Configuration
stock_list = ['PNB', 'BHEL', 'NTPC', 'BEL', 'IOC', 'TATASTEEL']
chosenmonths = ['Dec']
sort_by = 'ROI'  # 'Normal' or 'ROI'
atm_range_lower = 0.98  # Lower bound for ATM range (98% of current price)
atm_range_upper = 1.05  # Upper bound for ATM range (105% of current price)

print("✓ Libraries imported")
print(f"✓ Analyzing: {', '.join(stock_list)}")
print(f"✓ Expiry months: {', '.join(chosenmonths)}")
print(f"✓ ATM Range: {atm_range_lower*100:.0f}% - {atm_range_upper*100:.0f}% of current price")

✓ Libraries imported
✓ Analyzing: PNB, BHEL, NTPC, BEL, IOC, TATASTEEL
✓ Expiry months: Dec


In [3]:
# Initialize NSE Data Fetcher
fetcher = NSEDataFetcher()
print("✓ NSE API connected")

Using today's cached lot sizes
✓ Loaded 213 lot sizes from cache
✓ NSE API connected


In [ ]:
# Strategy 1: Short Straddle Analysis
# This runs continuously and refreshes every 15 seconds

while True:
    all_opportunities = []
    
    # Fetch data for each stock
    for symbol in stock_list:
        try:
            # Get current price and options
            current_price = fetcher.get_stock_price(symbol)
            lot_size = fetcher.get_lot_size(symbol)
            
            if current_price == 0:
                continue
            
            # Get options for chosen months
            for month in chosenmonths:
                options = fetcher.get_options_data(symbol, expiry_month=month)
                
                # Group options by strike price
                strikes = defaultdict(lambda: {'CE': None, 'PE': None})
                
                for opt in options:
                    strike = opt['strike']
                    opt_type = opt['option_type']
                    
                    # Only consider strikes within ATM range
                    if (atm_range_lower * current_price) <= strike <= (atm_range_upper * current_price):
                        strikes[strike][opt_type] = opt
                
                # Find strikes with both CALL (CE) and PUT (PE)
                for strike, options_pair in strikes.items():
                    ce_opt = options_pair['CE']
                    pe_opt = options_pair['PE']
                    
                    # Skip if either option is missing
                    if not ce_opt or not pe_opt:
                        continue
                    
                    # Extract data
                    call_premium = ce_opt['last_price']
                    put_premium = pe_opt['last_price']
                    call_volume = ce_opt['volume']
                    put_volume = pe_opt['volume']
                    expiry_full = ce_opt['expiry_date']
                    
                    # Format expiry date (remove year): "30-Dec-2025" -> "30-Dec"
                    expiry = '-'.join(expiry_full.split('-')[:2]) if expiry_full else ''
                    
                    # Skip if premiums are zero
                    if call_premium == 0 or put_premium == 0:
                        continue
                    
                    # Calculate metrics
                    combined_premium = call_premium + put_premium
                    investment = 0.50 * lot_size * current_price  # 50% margin
                    max_profit = combined_premium * lot_size
                    max_roi = (max_profit / investment) * 100
                    
                    # Safety ranges (breakeven points)
                    short_safety = strike - combined_premium  # Lower breakeven
                    long_safety = strike + combined_premium   # Upper breakeven
                    safety_range_width = long_safety - short_safety
                    
                    # Liquidity score (combined volume)
                    liquidity = call_volume + put_volume
                    
                    all_opportunities.append({
                        'Symbol': symbol,
                        'Current': current_price,
                        'Lot Size': lot_size,
                        'Strike': strike,
                        'Expiry': expiry,
                        'CALL': call_premium,
                        'PUT': put_premium,
                        'C+P': round(combined_premium, 2),
                        'Investment': int(investment),
                        'Max Profit': int(max_profit),
                        'ROI %': round(max_roi, 2),
                        'Short Safety': round(short_safety, 2),
                        'Long Safety': round(long_safety, 2),
                        'Safety Width': round(safety_range_width, 2),
                        'Liquidity': liquidity,
                        'CALL Vol': call_volume,
                        'PUT Vol': put_volume
                    })
        
        except Exception as e:
            print(f"⚠️  Error processing {symbol}: {e}")
    
    # Clear output and display results
    clear_output(wait=True)
    
    print("=" * 100)
    print(" " * 35 + "SHORT STRADDLE ANALYSIS")
    print("=" * 100)
    print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 100)
    
    # Create DataFrame
    if all_opportunities:
        df = pd.DataFrame(all_opportunities)
        
        # Sort by ROI (highest first) or normal
        if sort_by == 'ROI':
            df = df.sort_values('ROI %', ascending=False)
        else:
            df = df.sort_values(['Symbol', 'Strike'])
        
        df = df.reset_index(drop=True)
                
        display_cols = ['Symbol', 'Expiry', 'Current', 'Strike', 'Short Safety', 'Long Safety', 'Lot Size', 'CALL', 'PUT', 'C+P', 
                       'Investment', 'Max Profit', 'ROI %', 'CALL Vol', 'PUT Vol']
        print(df[display_cols].to_string(index=True))
    
    else:
        print("\n⚠️  No opportunities found. Market may be closed.")
    
    print("\n" + "=" * 100)
    print("Refreshing in 15 seconds... (Press ⏹️ to stop)")
    print("=" * 100)
    
    sleep(15)

                                   SHORT STRADDLE ANALYSIS
Last Updated: 2025-12-29 07:24:16
----------------------------------------------------------------------------------------------------

✅ Found 46 opportunities

       Symbol  Expiry  Current  Strike  Short Safety  Long Safety  Lot Size  CALL    PUT    C+P  Investment  Max Profit  ROI %  CALL Vol  PUT Vol
0         PNB  30-Dec   120.60   126.0        119.98       132.02      8000  0.02   6.00   6.02      482400       48160   9.98       327       13
1        BHEL  30-Dec   279.00   292.5        278.95       306.05      2625  0.05  13.50  13.55      366187       35568   9.71       540        5
2         IOC  30-Dec   162.10   170.0        162.23       177.77      4875  0.02   7.75   7.77      395118       37878   9.59       327       13
3   TATASTEEL  30-Dec   172.30   180.0        171.85       188.15      5500  0.05   8.10   8.15      473825       44825   9.46     11172      302
4        NTPC  30-Dec   325.65   340.0        324

KeyboardInterrupt: 